## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [281]:
# -*- coding: utf8 -*-
import codecs,glob
# import cellbell
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
import itertools as it
import pickle,pyperclip
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

from FrenchPhonology import makeFrench,setNeutralisation,normalizePhono,toipa,recoder
import SWiM_Utils as SwimU
import SWiM_Network as SwimN
from SWiM_Utils import *
from EspacesThematiques import etCols, sortCells


In [32]:
from ipywidgets import FloatProgress
from IPython.display import display

In [33]:
timeStamp="200909-1404"
timeStamp="Arbitrary"
timeStamp="3Cells"
repSamples=u"/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Jouets/"
fileSample=repSamples+"Jouet-%s.csv"%timeStamp
neutralMap="X"

In [34]:
import math
def rAn(r,n):
    f = math.factorial
    return f(n) / f(n-r)
def rCn(r,n):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

### Préparation des matrices de traits

In [35]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')
SwimU.fs=fs

In [36]:
validPhonemes=list(fs.supremum.concept.extent)
#for phoneme in validPhonemes:
#    print (phoneme, [phoneme], ";")

In [37]:
tableNeutralise=setNeutralisation(neutralMap)

### Vérification de la phonotactique des glides du français
- si *prononciation* est *None* renvoyer *None*
- ajout de diérèses dans les séquences mal-formées
- vérification des séquences consonne+glide à la finale

In [38]:
makeFrench("trw6ra",tableNeutralise), makeFrench("pEra,pEj6ra",tableNeutralise)

('trura', 'pEra,pEj6ra')

# Lecture du tableau de VERBES

### Lecture du lexique
- nomLexique pour le fichier
- names pour les noms de colonnes
- élimination des lignes dupliquées éventuelles (p.e. dépendre)

In [39]:
paradigmes=pd.read_csv(fileSample,sep=";",encoding="utf8")

In [40]:
if u"Unnamed: 0" in paradigmes:
    del paradigmes[u"Unnamed: 0"]

In [41]:
sampleCases=paradigmes.columns.tolist()
sampleCases.remove(u"lexeme")
sampleCases=sortCells(sampleCases)
SwimU.sampleCases=sampleCases
paradigmes=paradigmes[["lexeme"]+sampleCases]

In [333]:
print (len(paradigmes)*3)
print (paradigmes["pi3S inf ppMS".split(" ")].count().sum())
print (len(paradigmes.dropna(thresh=3)))

105
54
16


### Application de la neutralisation phonologique et stockage des paradigmes neutralisés correspondants

In [42]:
#Neutralize all the forms
for case in sampleCases:
#    print (case)
#    paradigmes[case]=paradigmes[case].apply(lambda x: recoder(x))
    paradigmes[case]=paradigmes[case].apply(lambda x: makeFrench(x,tableNeutralise))

In [43]:
def getWriterXLSX(fn):
    try:
        bookResult = load_workbook(fn)
        writerResult = pd.ExcelWriter(fn, engine='openpyxl')
        writerResult.book=bookResult
        writerResult.sheets = dict((ws.title, ws) for ws in bookResult.worksheets)
    except:
        writerResult = pd.ExcelWriter(fn, engine='openpyxl')
    return writerResult



In [44]:
def colorBlue(row):
    cellColors=[]
    for c,v in row.iteritems():
        print (c,v)
        if c=="lexeme":
            cellColors.append("")
        elif v==v:
            cellColors.append('background-color: #%X'%swimColorsHTML["data"])
        else:
            cellColors.append('background-color: #%X'%swimColorsHTML["UG"])
    print (cellColors)
    return cellColors

In [45]:
fileTrial=fileSample.replace(".csv","-%s.csv"%neutralMap)
paradigmes.to_csv(path_or_buf=fileTrial,sep=";",encoding="utf8",index=None)
fileTrialXLS=fileTrial.replace(".csv",".xlsx")
writerResult=getWriterXLSX(fileTrialXLS)
trialParadigme=paradigmes.style.apply(lambda x: colorBlue(x),axis=1)
trialParadigme.to_excel(writerResult,sheet_name="Sample")
writerResult.save()

lexeme aider
pi3S nan
inf Ede
ppMS nan
['', 'background-color: #FFFF80', 'background-color: #80FFFF', 'background-color: #FFFF80']
lexeme aimer
pi3S Em
inf nan
ppMS nan
['', 'background-color: #80FFFF', 'background-color: #FFFF80', 'background-color: #FFFF80']
lexeme arriver
pi3S ariv
inf nan
ppMS nan
['', 'background-color: #80FFFF', 'background-color: #FFFF80', 'background-color: #FFFF80']
lexeme chercher
pi3S nan
inf SErSe
ppMS nan
['', 'background-color: #FFFF80', 'background-color: #80FFFF', 'background-color: #FFFF80']
lexeme donner
pi3S nan
inf dOne
ppMS nan
['', 'background-color: #FFFF80', 'background-color: #80FFFF', 'background-color: #FFFF80']
lexeme fermer
pi3S fErm
inf nan
ppMS nan
['', 'background-color: #80FFFF', 'background-color: #FFFF80', 'background-color: #FFFF80']
lexeme laisser
pi3S lEs
inf lEse
ppMS nan
['', 'background-color: #80FFFF', 'background-color: #80FFFF', 'background-color: #FFFF80']
lexeme parler
pi3S parl
inf parle
ppMS nan
['', 'background-color: #8

In [46]:
bookResult = load_workbook(filename = fileTrialXLS)
writerResult = pd.ExcelWriter(fileTrialXLS, engine='openpyxl')
writerResult.book=bookResult
writerResult.sheets = dict((ws.title, ws) for ws in bookResult.worksheets)
ws=bookResult[u"Sample"]
from itertools import islice
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx, columns=cols)
normalizePhono(df,toipa).to_excel(writerResult,sheet_name=u"Sample")
writerResult.save()

### Boucle de calcul des analogies pour l'échantillon

In [47]:
# %%time

progressBar = FloatProgress(min=0, max=rCn(2,len(sampleCases)), description="Analogies (%d pairs)"%rCn(2,len(sampleCases)))
SwimU.progressBar=progressBar
display(progressBar)
debug=False
debug1=True
resultats=evaluerEchantillon(paradigmes)

FloatProgress(value=0.0, description=u'Analogies (3 pairs)', max=3.0)

In [48]:
classesFinales={}
for resultat in resultats:
    classesFinales[resultat]=resultats[resultat][0]
    classesFinales[(resultat[1],resultat[0])]=resultats[resultat][1]

In [49]:
fileRules=fileSample.replace(".csv","-%s.pkl"%neutralMap)
fileRules

u'/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Jouets/Jouet-3Cells-X.pkl'

In [50]:
with open(fileRules, 'wb') as output:
    pickle.dump(classesFinales, output, pickle.HIGHEST_PROTOCOL)

# Fin du traitement

# Combinaison des patrons flexionnels
## création d'un dictionnaire
    - key : paire de cases
    - values :
        - case:motif
        - case:motif
        

In [209]:
patterns={}
for c1,c2 in classesFinales:
    if c1<c2:
#         print (c1,c2)
        if (c1,c2) not in patterns:
            patterns[(c1,c2)]=[]
        for pf in classesFinales[(c1,c2)].classe:
            m1,m2=pf.split("-")
            m1="X"+m1
            m2="X"+m2
#             print (m1,m2)
            patterns[(c1,c2)].append({c1:m1,c2:m2})
#         print ()

In [207]:
# patron2pNum={}
# for paire, patrons in patterns.iteritems():
#     for patron in patrons:
#         print (patron)
#         pNum=raw_input("ref")
#         patron2pNum[frozenset(patron.items())]=pNum
pNum2patron={}
for p,pNum in patron2pNum.iteritems():
    pNum2patron[pNum]=p
patron2pNum
pNum2patron    

{'p1': frozenset({(u'inf', u'Xe'), (u'pi3S', u'X')}),
 'p10': frozenset({(u'inf', u'X6nir'), (u'pi3S', u'Xj\xea')}),
 'p11': frozenset({(u'inf', u'Xwar'), (u'ppMS', u'Xy')}),
 'p2': frozenset({(u'pi3S', u'X'), (u'ppMS', u'Xe')}),
 'p3': frozenset({(u'inf', 'X'), (u'ppMS', 'X')}),
 'p4': frozenset({(u'inf', u'Xale'), (u'pi3S', u'Xva')}),
 'p5': frozenset({(u'pi3S', u'Xwa'), (u'ppMS', u'Xy')}),
 'p6': frozenset({(u'inf', u'XEtr'), (u'ppMS', u'Xi')}),
 'p7': frozenset({(u'pi3S', u'X2'), (u'ppMS', u'Xy')}),
 'p8': frozenset({(u'inf', u'X\xe2dr'), (u'ppMS', u'Xi')}),
 'p9': frozenset({(u'inf', u'Xavwar'), (u'pi3S', u'XE')}),
 'pa1': frozenset({(u'inf', u'Xavwar'), (u'ppMS', u'Xy')}),
 'pa2': frozenset({(u'pi3S', u'Xa'), (u'ppMS', u'Xy')}),
 'pa3': frozenset({(u'inf', u'Xvwar'), (u'pi3S', u'X')}),
 'pe1': frozenset({(u'inf', u'Xr'), (u'ppMS', u'Xe')}),
 'pe2': frozenset({(u'pi3S', u'X'), (u'ppMS', u'Xte')}),
 'pe3': frozenset({(u'inf', u'Xtr'), (u'pi3S', u'X')}),
 'pf1': frozenset({(u'inf', 

## Combinaison des patrons

### combinaison de deux patrons

### combinaisons en boucle

In [215]:
def matchPF(source,cible):
    result={}
    pSource=patron2pNum[frozenset(source.items())]
    pCible=patron2pNum[frozenset(cible.items())]
    print(pSource,pCible,source[pivot],cible[pivot])
    s=source[pivot].replace("X","^(.*)")
    m=re.match(s,cible[pivot])
    if m:
        print("match12")
        print()
        result[pivot]=cible[pivot]
        result[pD1]=cible[pD1]
        sPrefix=m.group(1)
        result[pD0]=source[pD0].replace("X",sPrefix)
        result["combinaison"]="%s+%s"%(pSource,pCible)
        result["paires"]="%s-%s&%s-%s"%(p0[0],p0[1],p1[0],p1[1])
#         print (source,cible)
#         print ("=>",result)
#         print ()
    if source[pivot]!=cible[pivot]:
        s=cible[pivot].replace("X","^(.*)")
        m=re.match(s,source[pivot])
        if m:
            print("match21")
            print()
            result[pivot]=source[pivot]
            result[pD0]=source[pD0]
            sPrefix=m.group(1)
            result[pD1]=cible[pD1].replace("X",sPrefix)
            result["combinaison"]="%s+%s"%(pSource,pCible)
            result["paires"]="%s-%s&%s-%s"%(p0[0],p0[1],p1[0],p1[1])
    return result

patronsComplets=[]
for p0,p1 in it.combinations(patterns.keys(),2):
    if p0[0]==p1[0]:
        pivot=p0[0]
        pD0=p0[1]
        pD1=p1[1]
    elif p0[0]==p1[1]:
        pivot=p0[0]
        pD0=p0[1]
        pD1=p1[0]
    elif p0[1]==p1[0]:
        pivot=p0[1]
        pD0=p0[0]
        pD1=p1[1]
    else:
        pivot=p0[1]
        pD0=p0[0]
        pD1=p1[0]
        
#     print (p0,p1,pivot)
    for pf0 in patterns[p0]:
        for pf1 in patterns[p1]:
            patronsComplets.append(matchPF(pf0,pf1))
#         print()

patronsComplets=[d for d in patronsComplets if d!=dict()]
patronsComplets

p1 p6 Xe XEtr
p1 p3 Xe X
match21

p1 pa1 Xe Xavwar
p1 p11 Xe Xwar
p1 pe1 Xe Xr
p1 pf1 Xe Xr
p1 p8 Xe Xâdr
p9 p6 Xavwar XEtr
p9 p3 Xavwar X
match21

p9 pa1 Xavwar Xavwar
match12

p9 p11 Xavwar Xwar
match21

p9 pe1 Xavwar Xr
match21

p9 pf1 Xavwar Xr
match21

p9 p8 Xavwar Xâdr
pe3 p6 Xtr XEtr
match12

pe3 p3 Xtr X
match21

pe3 pa1 Xtr Xavwar
pe3 p11 Xtr Xwar
pe3 pe1 Xtr Xr
match21

pe3 pf1 Xtr Xr
match21

pe3 p8 Xtr Xâdr
pa3 p6 Xvwar XEtr
pa3 p3 Xvwar X
match21

pa3 pa1 Xvwar Xavwar
match12

pa3 p11 Xvwar Xwar
match21

pa3 pe1 Xvwar Xr
match21

pa3 pf1 Xvwar Xr
match21

pa3 p8 Xvwar Xâdr
p10 p6 X6nir XEtr
p10 p3 X6nir X
match21

p10 pa1 X6nir Xavwar
p10 p11 X6nir Xwar
p10 pe1 X6nir Xr
match21

p10 pf1 X6nir Xr
match21

p10 p8 X6nir Xâdr
pf3 p6 Xr XEtr
match12

pf3 p3 Xr X
match21

pf3 pa1 Xr Xavwar
match12

pf3 p11 Xr Xwar
match12

pf3 pe1 Xr Xr
match12

pf3 pf1 Xr Xr
match12

pf3 p8 Xr Xâdr
match12

p4 p6 Xale XEtr
p4 p3 Xale X
match21

p4 pa1 Xale Xavwar
p4 p11 Xale Xwar
p4 pe1 Xale Xr

[{'combinaison': 'p1+p3',
  u'inf': u'Xe',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'X',
  u'ppMS': u'Xe'},
 {'combinaison': 'p9+p3',
  u'inf': u'Xavwar',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xavwar'},
 {'combinaison': 'p9+pa1',
  u'inf': u'Xavwar',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xy'},
 {'combinaison': 'p9+p11',
  u'inf': u'Xavwar',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xavy'},
 {'combinaison': 'p9+pe1',
  u'inf': u'Xavwar',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xavwae'},
 {'combinaison': 'p9+pf1',
  u'inf': u'Xavwar',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xavwa'},
 {'combinaison': 'pe3+p6',
  u'inf': u'XEtr',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'XE',
  u'ppMS': u'Xi'},
 {'combinaison': 'pe3+p3',
  u'inf': u'Xtr',
  'paires': u'inf-pi3S&inf-ppMS',
  u'pi3S': u'X',
  u'ppMS': u'Xtr'},
 {'combinaison': 'pe3+pe1',
  u'inf': u'Xtr',
 

In [220]:
dfPatrons=pd.DataFrame(patronsComplets)#.dropna()
dfPatrons=dfPatrons.groupby(["inf","pi3S","ppMS"]).agg({"combinaison":{"nb":"count","combinaisons":", ".join}}).reset_index()
dfPatrons.columns = [b if b else a for a,b in dfPatrons.columns.to_flat_index()]
dfPatrons["nb combinaisons pi3S inf ppMS".split(" ")].to_excel("temp.xlsx")

In [294]:
# dfPatrons[['p1', 'p2',"p3"]]=pd.DataFrame(dfPatrons["combinaisons"].to_list(), columns=['p1', 'p2',"p3"])
# dfPatrons["c1"]=pd.Series(dfPatrons["p1"].str[0])["inf"]
# dfPatrons
dfPatrons#["p1"].str[0].apply(pd.Series)#.dropna()

,inf,pi3S,ppMS,nb,combinaisons
0,X,X,X,1,p3+pf2
1,X2e,X2,Xy,1,p1+p7
2,X2r,X2,Xy,1,pf3+p7
3,X2tr,X2,Xy,1,pe3+p7
4,X2vwar,X2,Xy,1,pa3+p7
5,X6nir,Xjê,X6ni,1,p10+pf1
6,X6nir,Xjê,X6nie,1,p10+pe1
7,X6nir,Xjê,X6nir,1,p10+p3
8,X6nir,Xjê,Xjê,1,p10+pf2
9,X6nir,Xjê,Xjêe,1,p10+p2


## Comparaison avec le plat de Stump & Finkel 2013

In [247]:
%store -r stemsSF2013
platSF=stemsSF2013.reset_index()
platSF.rename(columns={0:'lexeme'}, inplace=True)
platSF

,lexeme,stem01,stem02,stem03,stem04,stem05,stem06,stem07,stem08,stem09,...,stem10a,stem11,stem12,stem13,stem14,stem15,stem16,stem17,stem18,stem19
0,ÊTRE,-sɥi,-ɛ,!sɔm,!ɛt,-sɔ̃,-et,-etj,-fy,-fy,...,-səʁ,-swa,-swaj,-swa,-swaj,-swaj,-etʁ,-et,-ete,-ete
1,AVOIR,-ɛ,-a,-av,-av,-ɔ̃,-av,-avj,-y,-y,...,-oʁ,-ɛ,-ɛj,-ɛ,-ɛj,-ɛj,-avwaʁ,-ɛj,-y,-y
2,AIMER,-,-,-,-,-,-,-j,-ɛ,-a,...,-əʁ,-,-j,-,-,-,-e,-,-e,-e
3,COLLER,-ɔl,-ɔl,-ol,-ol,-ɔl,-ol,-olj,-olɛ,-ola,...,-oləʁ,-ɔl,-olj,-ɔl,-ol,-ol,-ole,-ol,-ole,-ole
4,BEURRER,-œʁ,-œʁ,-øʁ,-øʁ,-œʁ,-øʁ,-øʁj,-øʁɛ,-øʁa,...,-œʁəʁ,-œʁ,-øʁj,-œʁ,-øʁ,-øʁ,-øʁe,-øʁ,-øʁe,-øʁe
5,DÉJEUNER,-,-,-,-,-,-,-j,-ɛ,-a,...,-əʁ,-,-j,-,-,-,-e,-,-e,-e
6,ÉCROUER,-,-,-,-,-,-,-j,-ɛ,-a,...,-ʁ,-,-j,-,-,-,-e,-,-e,-e
7,ÉCHOUER,-u,-u,-w,-w,-u,-w,-uj,-wɛ,-wa,...,-uʁ,-u,-uj,-u,-w,-w,-we,-w,-we,-we
8,REFLUER,-,-,-,-,-,-,-j,-ɛ,-a,...,-ʁ,-,-j,-,-,-,-e,-,-e,-e
9,REMUER,-y,-y,-ɥ,-ɥ,-y,-ɥ,-yj,-ɥɛ,-ɥa,...,-yʁ,-y,-yj,-y,-ɥ,-ɥ,-ɥe,-ɥ,-ɥe,-ɥe


In [253]:
platSF=platSF["lexeme stem02 stem16 stem18 stem01".split(" ")].groupby("stem02 stem16 stem18".split(" ")).agg({"stem01":"count","lexeme":", ".join}).reset_index()
platSF["pi3S"]=platSF["stem02"].str.replace("-","X")
platSF["inf"]=platSF["stem16"].str.replace("-","X")
platSF["ppMS"]=platSF["stem18"].str.replace("-","X")
platSF["lexeme pi3S inf ppMS".split(" ")].to_excel("tempSF.xlsx")

In [313]:
def ipaNeutre(chaine):
    result=chaine.replace(u"ɔ",u"o")
    result=result.replace(u"ɛ",u"e")
    result=result.replace(u"œ",u"ø")
    return result

platCorrSF=pd.read_excel("tempSF.xlsx",index=None)
platCorrSF=platCorrSF["lexeme pi3S inf ppMS".split(" ")]
platCorrSF

,lexeme,pi3S,inf,ppMS
0,ALLER,Xva,Xale,Xale
1,HAÏR,Xɛ,Xaiʁ,Xai
2,RENDRE,X,Xdʁ,Xdy
3,MOUDRE,X,Xdʁ,Xly
4,COUDRE,X,Xdʁ,Xzy
5,"AIMER, DÉJEUNER, ÉCROUER, REFLUER",X,Xe,Xe
6,ASSAILLIR,X,Xiʁ,Xi
7,COURIR,X,Xiʁ,Xy
8,"APPUYER, BROYER",X,Xje,Xje
9,BOUILLIR,X,Xjiʁ,Xji


In [326]:
platRef=platCorrSF.to_dict(orient="row")
platRef={frozenset((k,ipaNeutre(v)) for k,v in p.items() if k!="lexeme"):p["lexeme"] for p in platRef}
print (len(platRef))
platRef

45


{frozenset({('inf', u'X\u0259ni\u0281'),
            ('pi3S', u'Xje\u0303'),
            ('ppMS', u'X\u0259ny')}): u'TENIR',
 frozenset({('inf', u'Xti\u0281'),
            ('pi3S', u'X'),
            ('ppMS', u'Xti')}): u'SENTIR',
 frozenset({('inf', u'Xje'), ('pi3S', u'X'), ('ppMS', u'Xje')}): u'ENVOYER',
 frozenset({('inf', u'Xiv\u0281'),
            ('pi3S', u'Xi'),
            ('ppMS', u'Xeky')}): u'VIVRE',
 frozenset({('inf', u'Xd\u0281'),
            ('pi3S', u'X'),
            ('ppMS', u'X')}): u'PEINDRE, JOINDRE, CRAINDRE',
 frozenset({('inf', u'Xd\u0281'),
            ('pi3S', u'X'),
            ('ppMS', u'Xdy')}): u'RENDRE',
 frozenset({('inf', u'Xje'), ('pi3S', u'Xi'), ('ppMS', u'Xje')}): u'COPIER',
 frozenset({('inf', u'Xe\u0281'),
            ('pi3S', u'Xe'),
            ('ppMS', u'Xy')}): u'PLAIRE',
 frozenset({('inf', u'Xavwa\u0281'),
            ('pi3S', u'Xe'),
            ('ppMS', u'Xy')}): u'SAVOIR',
 frozenset({('inf', u'Xt\u0281'),
            ('pi3S', u'X'),
     

In [315]:
def checkPatron(row):
    result=""
    matchRow=frozenset((k,ipaNeutre(row[k])) for k in "pi3S inf ppMS".split())
    if matchRow in platRef:
        result=platRef[matchRow]
        print (",".join(row["pi3S inf ppMS".split(" ")].tolist()))
        print (result,row.name)
    return result

In [316]:
dfMatch=dfPatrons.copy()
for col in "pi3S inf ppMS".split():
    dfMatch[col]=dfPatrons[col].apply(lambda x: recoder(x,toipa))
dfMatch.apply(checkPatron,axis=1)

Xɛ,Xɛtʁ,Xi
METTRE 11
Xva,Xale,Xale
ALLER 15
Xɛ,Xavwaʁ,Xy
SAVOIR 30
Xa,Xavwaʁ,Xy
AVOIR 31
X,Xe,Xe
CÉDER 36
X,Xʁ,X
FAIRE 39
X,Xtʁ,X
NAÎTRE 46
X,Xtʁ,Xte
ÊTRE 49
Xwa,Xwaʁ,Xy
VOIR, POURVOIR, CROIRE, BOIRE 61
Xwa,Xwatʁ,Xy
CROÎTRE 63


0                                  
1                                  
2                                  
3                                  
4                                  
5                                  
6                                  
7                                  
8                                  
9                                  
10                                 
11                           METTRE
12                                 
13                                 
14                                 
15                            ALLER
16                                 
17                                 
18                                 
19                                 
20                                 
21                                 
22                                 
23                                 
24                                 
25                                 
26                                 
27                          

In [ ]:
tabPaires=[]
for c1,c2 in it.combinations(sortCells(sampleCases),2):
    tabPaire={}
    print (c1,c2,end=" : ")
    try:
        ixLexeme=paradigmes.iloc[13:16][[c1,c2]].dropna(thresh=2).index.values[0]
        print (ixLexeme)
        exLexeme=paradigmes.loc[paradigmes.index==ixLexeme,"lexeme"].values[0]
        tabPaire["ligne"]=c1
        tabPaire["colonne"]=c2
        tabPaire["exemple"]=exLexeme
        tabPaires.append(tabPaire)
        print (c2,c1,end=" : ")
        print (ixLexeme)
        tabPaire={}
        tabPaire["ligne"]=c2
        tabPaire["colonne"]=c1
        tabPaire["exemple"]=exLexeme
        tabPaires.append(tabPaire)
    except:
        pass
display(pd.DataFrame(tabPaires))
crosstab=pd.pivot_table(pd.DataFrame(tabPaires),values="exemple",columns="colonne",index="ligne",aggfunc="first")
crosstab

In [ ]:
crosstab = crosstab.reindex(sortCells(crosstab.columns.tolist()), axis=1)
crosstab = crosstab.reindex(sortCells(crosstab.index.tolist()))
pyperclip.copy(crosstab.to_latex())

In [ ]:
patrons=classesFinales[("inf","pi2S")].patrons
for patron in patrons:
    m=re.match(ur"^[^.*\[\]()]*$",patrons[patron])
    if not m:
        print (patron,patrons[patron])